In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from prepare import telco_pipeline, telco_prep, get_telco_data, name_telco, drop_telco, fix_types_telco, fillna_telco, replace_telco, map_telco, reorder_telco, telco_train_val_test, dummies_telco, name_cols_telco, telco_modeling
#telco_work, telco_pipeline, rename_telco_proper, telco_prep, get_telco_data, name_telco, drop_telco, fix_types_telco, fillna_telco, replace_telco, map_telco, reorder_telco
from acquire import get_telco_data

In [2]:
#import telco data and remove:
# payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'
df = telco_work()
#confirm data
df.head()

NameError: name 'telco_work' is not defined

In [ ]:
#check total rows and columns
df.shape

(7043, 20)

In [ ]:
#review data for nulls and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  total_cha

In [ ]:
df = df.rename(columns={'payment_type': 'payment_method'})


Found 2 possible issues:

* Internet service type contains several nulls 
* Total charges should be a float but its currently an object

Lets first address the total charges. 

In [ ]:
%%skip
# Lets conveert the object to a float
df['total_charges'] = df['total_charges'].astype(float)

UsageError: Cell magic `%%skip` not found.


We were unable to convert the object into a float

Received ValueError: could not convert string to float: ' '

We need to investigate why there are blanks in this column

In [ ]:
# Lets use a boolean mask to discover 
df[df['total_charges'] == ' ']

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
945,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No,Two year,DSL,Credit card (automatic)
1731,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No,Two year,NaN,Mailed check
1906,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
2025,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.70,,No,One year,NaN,Mailed check
2176,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No,Two year,NaN,Mailed check
2250,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No,Two year,NaN,Mailed check
2855,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,73.35,,No,Two year,DSL,Mailed check
3052,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No,Two year,NaN,Mailed check
3118,Female,0,Yes,Yes,0,No,No phone service,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
4054,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No,Two year,DSL,Mailed check


We discovered that the reason for the blank values is due to new cutomers with no prior total charges. 

Lets replace the blanks with 0 and change the data type of the column

In [ ]:
#conveert blanks to 0 and change type to float
df['total_charges'] = df['total_charges'].replace(' ', 0).astype(float)

In [ ]:
#confirm data type has changed
print(df.dtypes)

gender                    object
senior_citizen             int64
partner                   object
dependents                object
tenure                     int64
phone_service             object
multiple_lines            object
online_security           object
online_backup             object
device_protection         object
tech_support              object
streaming_tv              object
streaming_movies          object
paperless_billing         object
monthly_charges          float64
total_charges            float64
churn                     object
contract_type             object
internet_service_type     object
payment_method            object
dtype: object


Lets now tackle the nulls in the internet_service_type column
first lets find out how many are there

In [ ]:
#find any nulls in the data
df.isna().sum()

gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type    1526
payment_method              0
dtype: int64

Seems like internet sercice has 1500 nulls. 
lets investigate why

In [ ]:
df[df.internet_service_type.isna()].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
20,Female,1,Yes,No,50,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.20,1306.30,No,One year,NaN,Electronic check
23,Female,0,No,No,3,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,57.20,No,Month-to-month,NaN,Mailed check
24,Female,0,Yes,Yes,4,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,Month-to-month,NaN,Mailed check
27,Male,0,Yes,Yes,54,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.40,1090.60,No,Two year,NaN,Credit card (automatic)
28,Male,0,No,No,26,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.60,471.85,No,One year,NaN,Bank transfer (automatic)


In [ ]:
df.shape

(7043, 20)

It was null because the customer has no internet service.
Lets replace the null values with no internet service

In [ ]:
df['internet_service_type'] = df['internet_service_type'].fillna("No internet service")

In [ ]:
#confirm no nulls, and categorical options
print(df['internet_service_type'].isna().sum())
print(df['internet_service_type'].unique())

0
['DSL' 'Fiber optic' 'No internet service']


Lets check the unique values of evertying else to try and identify other possible issues

In [ ]:
# First lets get the names of all the columsn
df.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'contract_type', 'internet_service_type', 'payment_method'],
      dtype='object')

In [ ]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
                    'phone_service', 'multiple_lines', 'online_security', 'online_backup',
                    'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                    'paperless_billing', 'churn',
                    'contract_type', 'internet_service_type', 'payment_method']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         [0 1]
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes' 'No phone service']
online_security        ['No' 'Yes' 'No internet service']
online_backup          ['Yes' 'No' 'No internet service']
device_protection      ['No' 'Yes' 'No internet service']
tech_support           ['Yes' 'No' 'No internet service']
streaming_tv           ['Yes' 'No' 'No internet service']
streaming_movies       ['No' 'Yes' 'No internet service']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_method         ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


Lets change the senior_citizen column from 0, 1 to true / false


In [ ]:
df.shape

(7043, 20)

In [ ]:
df['senior_citizen'] = df['senior_citizen'].replace({1: 'Yes', 0: 'No'})

In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,Female,No,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check
1,Male,No,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
2,Male,No,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,Yes,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,Yes,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check


We have no categorical data for internet service like we do for phone service. lets create a new column

In [ ]:
df['internet_service'] = df['internet_service_type'].map({
'DSL': 'Yes',
'Fiber optic': 'Yes',
'No internet service': 'No'
})

In [ ]:
df.internet_service.unique()

array(['Yes', 'No'], dtype=object)

In [ ]:
df['internet_service_type'].unique()

array(['DSL', 'Fiber optic', 'No internet service'], dtype=object)

In [ ]:
df.shape

(7043, 21)

Lets confirm that in fact we no internet service was assigned correctly

In [ ]:
# Lets increase the max columns since we have over 20
pd.set_option('display.max_columns', None)

In [ ]:
df[df['internet_service'] == 'No'].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method,internet_service
20,Female,Yes,Yes,No,50,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.20,1306.30,No,One year,No internet service,Electronic check,No
23,Female,No,No,No,3,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,57.20,No,Month-to-month,No internet service,Mailed check,No
24,Female,No,Yes,Yes,4,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,Month-to-month,No internet service,Mailed check,No
27,Male,No,Yes,Yes,54,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.40,1090.60,No,Two year,No internet service,Credit card (automatic),No
28,Male,No,No,No,26,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.60,471.85,No,One year,No internet service,Bank transfer (automatic),No


Lets ensure that the bi categorical columns only contain 2 categories

multiple_lines "No phone service" to No

online_security, online_backup, device_protection, tech_support, streaming_tv, streaming_movies  "No internet service" to No

We will start with multiple lines first

In [ ]:
df[df['multiple_lines'] == 'No phone service'].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method,internet_service
10,Male,No,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic),Yes
14,Female,No,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.10,No,Two year,DSL,Bank transfer (automatic),Yes
16,Female,No,Yes,Yes,34,No,No phone service,Yes,No,Yes,Yes,Yes,Yes,No,61.25,1993.20,No,One year,DSL,Mailed check,Yes
19,Male,Yes,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.10,25.10,Yes,Month-to-month,DSL,Electronic check,Yes
25,Female,No,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.50,30.50,Yes,Month-to-month,DSL,Bank transfer (automatic),Yes


In [ ]:
df['multiple_lines'] = df['multiple_lines'].replace('No phone service', 'No')

Confirm that we have 0 "no phone sercive" 

In [ ]:
df.multiple_lines.unique()

array(['No', 'Yes'], dtype=object)

In [ ]:
df.shape

(7043, 21)

We filter by no internet sercice on treaming movies so we can make sure our changes take effect

In [ ]:
df[df['streaming_movies'] == 'No internet service'].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method,internet_service
20,Female,Yes,Yes,No,50,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.20,1306.30,No,One year,No internet service,Electronic check,No
23,Female,No,No,No,3,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,57.20,No,Month-to-month,No internet service,Mailed check,No
24,Female,No,Yes,Yes,4,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,Month-to-month,No internet service,Mailed check,No
27,Male,No,Yes,Yes,54,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.40,1090.60,No,Two year,No internet service,Credit card (automatic),No
28,Male,No,No,No,26,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.60,471.85,No,One year,No internet service,Bank transfer (automatic),No


Lets change these 5 columns and then reload the table with the same filter

In [ ]:
df['online_security'] = df['online_security'].replace('No internet service', 'No')
df['online_backup'] = df['online_security'].replace('No internet service', 'No')
df['device_protection'] = df['online_security'].replace('No internet service', 'No')
df['tech_support'] = df['online_security'].replace('No internet service', 'No')
df['streaming_tv'] = df['online_security'].replace('No internet service', 'No')

In [ ]:
df[df['streaming_movies'] == 'No internet service'].head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method,internet_service
20,Female,Yes,Yes,No,50,Yes,Yes,No,No,No,No,No,No internet service,No,25.20,1306.30,No,One year,No internet service,Electronic check,No
23,Female,No,No,No,3,Yes,No,No,No,No,No,No,No internet service,No,19.85,57.20,No,Month-to-month,No internet service,Mailed check,No
24,Female,No,Yes,Yes,4,Yes,No,No,No,No,No,No,No internet service,No,20.35,76.35,Yes,Month-to-month,No internet service,Mailed check,No
27,Male,No,Yes,Yes,54,Yes,No,No,No,No,No,No,No internet service,No,20.40,1090.60,No,Two year,No internet service,Credit card (automatic),No
28,Male,No,No,No,26,Yes,No,No,No,No,No,No,No internet service,Yes,19.60,471.85,No,One year,No internet service,Bank transfer (automatic),No


As you can see above those columns did change to 'no'

Lets change the last column since we know our code works as expected

In [ ]:
df.shape

(7043, 21)

In [ ]:
df['streaming_movies'] = df['online_security'].replace('No internet service', 'No')

Lets see what categorical data is left

In [ ]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
                    'phone_service', 'multiple_lines', 'online_security', 'online_backup',
                    'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                    'paperless_billing', 'churn',
                    'contract_type', 'internet_service_type', 'payment_method']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         ['No' 'Yes']
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes']
online_security        ['No' 'Yes']
online_backup          ['No' 'Yes']
device_protection      ['No' 'Yes']
tech_support           ['No' 'Yes']
streaming_tv           ['No' 'Yes']
streaming_movies       ['No' 'Yes']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_method         ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


Seems like we can use a new column for categorizing manual and automatic payments. 

lets first change the names of the category options and the name of the column to payment method. After we will proceed with creating the new column. 

In [ ]:
df.shape

(7043, 21)

In [ ]:
df['payment_method'] = df['payment_method'].replace({
    'Credit card (automatic)': 'Credit card',
    'Bank transfer (automatic)': 'Bank transfer'
})

In [ ]:
df.payment_method.unique()

array(['Mailed check', 'Electronic check', 'Credit card', 'Bank transfer'],
      dtype=object)

In [ ]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
                    'phone_service', 'multiple_lines', 'online_security', 'online_backup',
                    'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                    'paperless_billing', 'churn',
                    'contract_type', 'internet_service_type', 'payment_method']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         ['No' 'Yes']
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes']
online_security        ['No' 'Yes']
online_backup          ['No' 'Yes']
device_protection      ['No' 'Yes']
tech_support           ['No' 'Yes']
streaming_tv           ['No' 'Yes']
streaming_movies       ['No' 'Yes']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_method         ['Mailed check' 'Electronic check' 'Credit card' 'Bank transfer']


Lets create the new column based on the payment method column

In [ ]:
payment_mapping = {
    'Mailed check': 'Manual',
    'Electronic check': 'Manual',
    'Credit card': 'Automatic',
    'Bank transfer': 'Automatic'
}

df['payment_type'] = df['payment_method'].map(payment_mapping)

In [ ]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'churn',
       'contract_type', 'internet_service_type', 'payment_method',
       'internet_service', 'payment_type']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         ['No' 'Yes']
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes']
online_security        ['No' 'Yes']
online_backup          ['No' 'Yes']
device_protection      ['No' 'Yes']
tech_support           ['No' 'Yes']
streaming_tv           ['No' 'Yes']
streaming_movies       ['No' 'Yes']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_method         ['Mailed check' 'Electronic check' 'Credit card' 'Bank transfer']
internet_service       ['Yes' 'No']
payment_type           ['Manual' 'Automatic']


In [ ]:
df.shape

(7043, 22)

Seems like we could use another categorical column for contract type

In [ ]:
df['contract'] = df['contract_type'].map({
    'One year': 'Yes',
    'Two year': 'Yes',
    'Month-to-month': 'No'
})

In [ ]:
# We assign the list of column names to a variable. Lets removed columns that hold numerical values. 
columns_to_check = ['gender', 'senior_citizen', 'partner', 'dependents',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'churn',
       'contract_type', 'internet_service_type', 'payment_method',
       'internet_service', 'payment_type', 'contract']

# Calculate the maximum column name length for formatting
max_col_length = max(len(column) for column in columns_to_check)

# Loop through the columns and print unique values with aligned columns
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"{column.ljust(max_col_length)}  {unique_values}")

gender                 ['Female' 'Male']
senior_citizen         ['No' 'Yes']
partner                ['Yes' 'No']
dependents             ['Yes' 'No']
phone_service          ['Yes' 'No']
multiple_lines         ['No' 'Yes']
online_security        ['No' 'Yes']
online_backup          ['No' 'Yes']
device_protection      ['No' 'Yes']
tech_support           ['No' 'Yes']
streaming_tv           ['No' 'Yes']
streaming_movies       ['No' 'Yes']
paperless_billing      ['Yes' 'No']
churn                  ['No' 'Yes']
contract_type          ['One year' 'Month-to-month' 'Two year']
internet_service_type  ['DSL' 'Fiber optic' 'No internet service']
payment_method         ['Mailed check' 'Electronic check' 'Credit card' 'Bank transfer']
internet_service       ['Yes' 'No']
payment_type           ['Manual' 'Automatic']
contract               ['Yes' 'No']


In [ ]:
df.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'contract_type', 'internet_service_type', 'payment_method',
       'internet_service', 'payment_type', 'contract'],
      dtype='object')

In [ ]:
new_column_order = [
    'gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
    'phone_service', 'multiple_lines', 'internet_service', 'internet_service_type', 'online_security', 'online_backup',
    'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract', 'contract_type', 'payment_method', 'payment_type',
    'paperless_billing', 'monthly_charges', 'total_charges', 'churn'
]

df = df[new_column_order]

In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,internet_service_type,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,contract_type,payment_method,payment_type,paperless_billing,monthly_charges,total_charges,churn
0,Female,No,Yes,Yes,9,Yes,No,Yes,DSL,No,No,No,No,No,No,Yes,One year,Mailed check,Manual,Yes,65.6,593.30,No
1,Male,No,No,No,9,Yes,Yes,Yes,DSL,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,No,59.9,542.40,No
2,Male,No,No,No,4,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,73.9,280.85,Yes
3,Male,Yes,Yes,No,13,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,98.0,1237.85,Yes
4,Female,Yes,Yes,No,3,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,Yes,83.9,267.40,Yes


In [ ]:
df.shape

(7043, 23)

In [ ]:
df = telco_prep()

In [ ]:
df.shape

(7043, 23)

In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,internet_service_type,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,contract_type,payment_method,payment_type,paperless_billing,monthly_charges,total_charges,churn
0,Female,No,Yes,Yes,9,Yes,No,Yes,DSL,No,No,No,No,No,No,Yes,One year,Mailed check,Manual,Yes,65.6,593.30,No
1,Male,No,No,No,9,Yes,Yes,Yes,DSL,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,No,59.9,542.40,No
2,Male,No,No,No,4,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,73.9,280.85,Yes
3,Male,Yes,Yes,No,13,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,98.0,1237.85,Yes
4,Female,Yes,Yes,No,3,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,Yes,83.9,267.40,Yes


In [ ]:
df = get_telco_data()


In [ ]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:
def rename_telco_new(df):
    print("Before renaming:", df.columns)
    
    # rename column to something more appropriate
    df = df.rename(columns={'payment_type': 'payment_method'})
    
    print("After renaming:", df.columns)
    
    return df


In [ ]:
df = name_telco(df)

In [ ]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:
df = df.rename(columns={'payment_type': 'payment_method'})


In [ ]:

df = drop_telco(df)


In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:

df = fix_types_telco(df)


In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:

df = fillna_telco(df)


In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:

df = replace_telco(df)


In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method
0,Female,No,Yes,Yes,9,Yes,No,No,No,No,No,No,No,Yes,65.6,593.30,No,One year,DSL,Mailed check
1,Male,No,No,No,9,Yes,Yes,No,No,No,No,No,No,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
2,Male,No,No,No,4,Yes,No,No,No,No,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,Yes,Yes,No,13,Yes,No,No,No,No,No,No,No,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,Yes,Yes,No,3,Yes,No,No,No,No,No,No,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check


In [ ]:

df = map_telco(df)


In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_method,internet_service,payment_type,contract
0,Female,No,Yes,Yes,9,Yes,No,No,No,No,No,No,No,Yes,65.6,593.30,No,One year,DSL,Mailed check,Yes,Manual,Yes
1,Male,No,No,No,9,Yes,Yes,No,No,No,No,No,No,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,Yes,Manual,No
2,Male,No,No,No,4,Yes,No,No,No,No,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check,Yes,Manual,No
3,Male,Yes,Yes,No,13,Yes,No,No,No,No,No,No,No,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check,Yes,Manual,No
4,Female,Yes,Yes,No,3,Yes,No,No,No,No,No,No,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check,Yes,Manual,No


In [ ]:

df = reorder_telco(df)

In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,internet_service_type,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,contract_type,payment_method,payment_type,paperless_billing,monthly_charges,total_charges,churn
0,Female,No,Yes,Yes,9,Yes,No,Yes,DSL,No,No,No,No,No,No,Yes,One year,Mailed check,Manual,Yes,65.6,593.30,No
1,Male,No,No,No,9,Yes,Yes,Yes,DSL,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,No,59.9,542.40,No
2,Male,No,No,No,4,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,73.9,280.85,Yes
3,Male,Yes,Yes,No,13,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Electronic check,Manual,Yes,98.0,1237.85,Yes
4,Female,Yes,Yes,No,3,Yes,No,Yes,Fiber optic,No,No,No,No,No,No,No,Month-to-month,Mailed check,Manual,Yes,83.9,267.40,Yes


In [ ]:
#create the dummies
df = pd.get_dummies(df)

In [ ]:
#check the column names, identify dropable columns
df.columns

Index(['tenure', 'monthly_charges', 'total_charges', 'gender_Female',
       'gender_Male', 'senior_citizen_No', 'senior_citizen_Yes', 'partner_No',
       'partner_Yes', 'dependents_No', 'dependents_Yes', 'phone_service_No',
       'phone_service_Yes', 'multiple_lines_No', 'multiple_lines_Yes',
       'internet_service_No', 'internet_service_Yes',
       'internet_service_type_DSL', 'internet_service_type_Fiber optic',
       'internet_service_type_No internet service', 'online_security_No',
       'online_security_Yes', 'online_backup_No', 'online_backup_Yes',
       'device_protection_No', 'device_protection_Yes', 'tech_support_No',
       'tech_support_Yes', 'streaming_tv_No', 'streaming_tv_Yes',
       'streaming_movies_No', 'streaming_movies_Yes', 'contract_No',
       'contract_Yes', 'contract_type_Month-to-month',
       'contract_type_One year', 'contract_type_Two year',
       'payment_method_Bank transfer', 'payment_method_Credit card',
       'payment_method_Electronic chec

In [ ]:
# Drop unwanted columns
df = df.drop(
                        columns= ['gender_Female', 'partner_No', 'dependents_No', 'phone_service_No', 'multiple_lines_No', 
                                    'multiple_lines_No phone service', 'online_security_No', 'online_security_No internet service', 
                                    'online_backup_No', 'online_backup_No internet service', 'device_protection_No', 'device_protection_No internet service', 
                                    'tech_support_No', 'tech_support_No internet service', 'streaming_tv_No', 'streaming_tv_No internet service', 
                                    'streaming_movies_No', 'streaming_movies_No internet service', 'paperless_billing_No', 'churn_No', 
                                    'internet_service_type_No internet service'])

KeyError: "['multiple_lines_No phone service', 'online_security_No internet service', 'online_backup_No internet service', 'device_protection_No internet service', 'tech_support_No internet service', 'streaming_tv_No internet service', 'streaming_movies_No internet service'] not found in axis"

In [ ]:
#check columns to confirm only wanted remains
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
#lets rename our columns to something more legible

column_mapping = {
    'senior_citizen': 'senior_citizen',
    'tenure': 'tenure',
    'monthly_charges': 'monthly_charges',
    'total_charges': 'total_charges',
    'gender_Male': 'male',
    'partner_Yes': 'partner',
    'dependents_Yes': 'dependents',
    'phone_service_Yes': 'phone_service',
    'multiple_lines_Yes': 'multiple_lines',
    'online_security_Yes': 'online_security',
    'online_backup_Yes': 'online_backup',
    'device_protection_Yes': 'device_protection',
    'tech_support_Yes': 'tech_support',
    'streaming_tv_Yes': 'streaming_tv',
    'streaming_movies_Yes': 'streaming_movies',
    'paperless_billing_Yes': 'paperless_billing',
    'churn_Yes': 'churn',
    'contract_type_Month-to-month': 'contract_month',
    'contract_type_One year': 'contract_one_year',
    'contract_type_Two year': 'contract_two_year',
    'internet_service_type_DSL': 'internet_dsl',
    'internet_service_type_Fiber optic': 'internet_fiber_optic',
    'payment_type_Bank transfer (automatic)': 'payment_bank_transfer',
    'payment_type_Credit card (automatic)': 'payment_credit_card',
    'payment_type_Electronic check': 'payment_electronic_check',
    'payment_type_Mailed check': 'payment_mailed_check'
}

    # Rename the columns using the mapping
df.rename(columns=column_mapping, inplace=True)

# Print the new column names
print(df.columns)

In [ ]:
df.head()

In [ ]:
#review data for nulls and data types
df.info()

In [ ]:
train, val, test = telco_pipeline()

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train = rename_telco_proper(train)

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
df = telco_prep()

In [ ]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,internet_service_type,online_security,...,streaming_tv,streaming_movies,contract,contract_type,payment_method,payment_type,paperless_billing,monthly_charges,total_charges,churn
0,Female,No,Yes,Yes,9,Yes,No,Yes,DSL,No,...,No,No,Yes,One year,Mailed check,Manual,Yes,65.6,593.30,No
1,Male,No,No,No,9,Yes,Yes,Yes,DSL,No,...,No,No,No,Month-to-month,Mailed check,Manual,No,59.9,542.40,No
2,Male,No,No,No,4,Yes,No,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Electronic check,Manual,Yes,73.9,280.85,Yes
3,Male,Yes,Yes,No,13,Yes,No,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Electronic check,Manual,Yes,98.0,1237.85,Yes
4,Female,Yes,Yes,No,3,Yes,No,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Mailed check,Manual,Yes,83.9,267.40,Yes


In [ ]:
train, val, test = telco_train_val_test(df, 'churn')

In [ ]:
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,internet_service_type,online_security,...,streaming_tv,streaming_movies,contract,contract_type,payment_method,payment_type,paperless_billing,monthly_charges,total_charges,churn
5609,Male,No,No,No,14,Yes,No,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Electronic check,Manual,No,76.45,1117.55,No
2209,Male,No,No,No,5,Yes,No,Yes,DSL,No,...,No,No,Yes,One year,Mailed check,Manual,Yes,70.00,347.40,Yes
6919,Male,No,Yes,No,35,Yes,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Electronic check,Manual,Yes,75.20,2576.20,Yes
2284,Male,No,Yes,No,58,Yes,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Two year,Electronic check,Manual,Yes,86.10,4890.50,No
845,Female,No,No,No,2,Yes,No,Yes,DSL,No,...,No,No,No,Month-to-month,Mailed check,Manual,Yes,49.60,114.70,Yes


In [ ]:
train = pd.get_dummies(train)

In [ ]:
train.head()

,tenure,monthly_charges,total_charges,gender_Female,gender_Male,senior_citizen_No,senior_citizen_Yes,partner_No,partner_Yes,dependents_No,...,payment_method_Bank transfer,payment_method_Credit card,payment_method_Electronic check,payment_method_Mailed check,payment_type_Automatic,payment_type_Manual,paperless_billing_No,paperless_billing_Yes,churn_No,churn_Yes
5609,14,76.45,1117.55,False,True,True,False,True,False,True,...,False,False,True,False,False,True,True,False,True,False
2209,5,70.00,347.40,False,True,True,False,True,False,True,...,False,False,False,True,False,True,False,True,False,True
6919,35,75.20,2576.20,False,True,True,False,False,True,True,...,False,False,True,False,False,True,False,True,False,True
2284,58,86.10,4890.50,False,True,True,False,False,True,True,...,False,False,True,False,False,True,False,True,True,False
845,2,49.60,114.70,True,False,True,False,True,False,True,...,False,False,False,True,False,True,False,True,False,True


In [ ]:
train.columns

Index(['tenure', 'monthly_charges', 'total_charges', 'gender_Female',
       'gender_Male', 'senior_citizen_No', 'senior_citizen_Yes', 'partner_No',
       'partner_Yes', 'dependents_No', 'dependents_Yes', 'phone_service_No',
       'phone_service_Yes', 'multiple_lines_No', 'multiple_lines_Yes',
       'internet_service_No', 'internet_service_Yes',
       'internet_service_type_DSL', 'internet_service_type_Fiber optic',
       'internet_service_type_No internet service', 'online_security_No',
       'online_security_Yes', 'online_backup_No', 'online_backup_Yes',
       'device_protection_No', 'device_protection_Yes', 'tech_support_No',
       'tech_support_Yes', 'streaming_tv_No', 'streaming_tv_Yes',
       'streaming_movies_No', 'streaming_movies_Yes', 'contract_No',
       'contract_Yes', 'contract_type_Month-to-month',
       'contract_type_One year', 'contract_type_Two year',
       'payment_method_Bank transfer', 'payment_method_Credit card',
       'payment_method_Electronic chec

In [ ]:
keep_cols = ['tenure', 'monthly_charges', 'internet_service_type_DSL', 'internet_service_type_Fiber optic', 
             'internet_service_type_No internet service', 'payment_method_Bank transfer', 
             'payment_method_Credit card', 'payment_method_Electronic check', 'payment_method_Mailed check', 'churn_Yes']

In [ ]:
keep_cols = ['tenure', 'monthly_charges', 'gender_Male', 'senior_citizen_Yes',
        'partner_Yes', 'dependents_Yes', 'phone_service_Yes', 'multiple_lines_Yes', 
        'internet_service_Yes','internet_service_type_DSL', 'internet_service_type_Fiber optic',
        'internet_service_type_No internet service', 'tech_support_Yes', 'contract_Yes',
        'contract_type_One year', 'contract_type_Two year', 'payment_method_Bank transfer', 
        'payment_method_Credit card', 'payment_method_Electronic check', 'payment_method_Mailed check',
        'payment_type_Automatic', 'payment_type_Manual', 'paperless_billing_Yes', 'churn_Yes']

In [ ]:
train = train[keep_cols]

In [ ]:
train.head()

,tenure,monthly_charges,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_No internet service,payment_method_Bank transfer,payment_method_Credit card,payment_method_Electronic check,payment_method_Mailed check,churn_Yes
5609,14,76.45,False,True,False,False,False,True,False,False
2209,5,70.00,True,False,False,False,False,False,True,True
6919,35,75.20,False,True,False,False,False,True,False,True
2284,58,86.10,True,False,False,False,False,True,False,False
845,2,49.60,True,False,False,False,False,False,True,True


In [ ]:
train.columns

Index(['tenure', 'monthly_charges', 'internet_service_type_DSL',
       'internet_service_type_Fiber optic',
       'internet_service_type_No internet service',
       'payment_method_Bank transfer', 'payment_method_Credit card',
       'payment_method_Electronic check', 'payment_method_Mailed check',
       'churn_Yes'],
      dtype='object')

In [ ]:
train, val, test = telco_pipeline()

In [ ]:
train.head()

,tenure,monthly_charges,internet_dsl,internet_fiber_optics,internet_no_internet,payment_method_bank_transfer,payment_method_credit_card,payment_method_electronic_check,payment_method_mailed_check,churn
5609,14,76.45,False,True,False,False,False,True,False,False
2209,5,70.00,True,False,False,False,False,False,True,True
6919,35,75.20,False,True,False,False,False,True,False,True
2284,58,86.10,True,False,False,False,False,True,False,False
845,2,49.60,True,False,False,False,False,False,True,True


In [ ]:
train_X, val_X, test_X, train_y, val_y, test_y = modeling_telco(train, val, test)

In [ ]:
train_X.head()

,tenure,monthly_charges,internet_dsl,internet_fiber_optics,internet_no_internet,payment_method_bank_transfer,payment_method_credit_card,payment_method_electronic_check,payment_method_mailed_check
0,14,76.45,False,True,False,False,False,True,False
1,5,70.00,True,False,False,False,False,False,True
2,35,75.20,False,True,False,False,False,True,False
3,58,86.10,True,False,False,False,False,True,False
4,2,49.60,True,False,False,False,False,False,True


In [ ]:
train_X.shape, val_X.shape, test_X.shape, train_y.shape, val_y.shape, test_y.shape

((4930, 9), (1056, 9), (1057, 9), (4930, 1), (1056, 1), (1057, 1))

In [ ]:
train_X, val_X, test_X, train_y, val_y, test_y = telco_modeling()

In [ ]:
train_X.head()

,tenure,monthly_charges,internet_dsl,internet_fiber_optics,internet_no_internet,payment_method_bank_transfer,payment_method_credit_card,payment_method_electronic_check,payment_method_mailed_check
0,14,76.45,False,True,False,False,False,True,False
1,5,70.00,True,False,False,False,False,False,True
2,35,75.20,False,True,False,False,False,True,False
3,58,86.10,True,False,False,False,False,True,False
4,2,49.60,True,False,False,False,False,False,True


In [ ]:
train_y.head()

,churn
0,False
1,True
2,True
3,False
4,True


In [ ]:
train_X.shape, val_X.shape, test_X.shape, train_y.shape, val_y.shape, test_y.shape

((4930, 9), (1056, 9), (1057, 9), (4930, 1), (1056, 1), (1057, 1))